<a href="https://colab.research.google.com/github/sethcoast/attention/blob/main/attention_is_all_you_need.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
# what does the architecture look like?

# How has Pytorch changed in the last little while?

# What do I need for all of this?
# 1. Training data
# 2. Model
# 3. Training loop

# This will likely be done in a few separate phases
# 1. Build and validate the model architecture
        # training data?
# 2. Set up training/snapshotting system in GCP


In [11]:
# Import the shit
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torchvision
import torchvision.transforms

import numpy as np
import matplotlib.pyplot as plt


In [15]:
# get the data?


In [18]:
# Feed forward block (sublayer)
class FeedForwardBlock(nn.Module):
    def __init__(self, dmodel):
        super(FeedForwardBlock, self).__init__()
        # Define the feed forward network layers
        self.fc1 = nn.Linear(dmodel, dmodel)
        self.fc2 = nn.Linear(dmodel, dmodel)
        self.dropout = nn.Dropout(0.1)
        self.relu = nn.ReLU(inplace=True) # inplace = True to save memory

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.dropout(x)

        return x

# Multihead Attention block (Sublayer)
class MultiheadAttentionBlock(nn.Module):
    def __init__(self, dmodel, mask=False):
        super(MultiheadAttentionBlock, self).__init__()
        self.mhAtt = nn.MultiheadAttention(dmodel, 8)
        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        x = self.mhAtt(x)
        x = self.dropout(x)

        return x


In [13]:
# define the encoder model
class Encoder(nn.Module):
    def __init__(self, dmodel):
        super(Encoder, self).__init__()
        # Define the attention layer
        self.mhaBlock = MultiheadAttentionBlock(dmodel)
        self.ff = FeedForwardBlock(dmodel)


    def forward(self, x):
        # Multi Head Attention Sublayer()
        out = self.mhaBlock(x)
        out += x # residual connection
        out = self.norm(out) # normalization

        # Feed Forward Sublayer()
        out2 = self.ff(out)
        out2 += out # residual connection
        out2 = self.norm(out2) # normalization

        return out2


In [14]:
class TransformerNet(nn.Module):
    def __init__(self, dmodel):
        super(TransformerNet, self).__init__()
        # Define all of the stuff needed for the Decoder network
        # todo: masked? How to do that? leave for v2?
        self.mhAtt = nn.MultiheadAttention(dmodel, 8)
        # Define the second fully connected layer
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Apply a ReLU non-linearity to the first layer's output
        x = F.relu(self.fc1(x))
        # The output layer with no non-linearity for classification
        x = self.fc2(x)
        return x
